In [1]:
import os
import sys

sys.path.append(os.path.abspath('..'))

import numpy as np
import matplotlib.pyplot as plt
import idx2numpy
import pandas as pd
from lib.Net import *
from lib.Func import *

In [2]:
# (batch_size, 1, 28, 28) -> (batch_size, 2, 24, 24) -> (batch_size, 2*24*24) -> (batch_size, 64) -> (batch_size, 10)
architecture = [
    {"module": Conv2d, "params": {"in_channels": 1, "out_channels": 2, "kernel_size": 5}},
    {"module": Flatten},
    {"module": Mlp, "params": {"input_dim": 1152, "output_dim": 64, "activation": "gelu"}},
    {"module": Dropout, "params": {"p": 0.2}},
    {"module": Mlp, "params": {"input_dim": 64, "output_dim": 10, "activation": "softmax"}},
]

In [3]:
net = Net(architecture)

In [4]:
datapath = "../data/MNIST/train-images.idx3-ubyte"
labelpath = "../data/MNIST/train-labels.idx1-ubyte"
modelpath = "../model/task2/MNIST.json"

In [ ]:
data = idx2numpy.convert_from_file(datapath)    # (60000, 28, 28)
data = np.expand_dims(data, axis=1)             # 添加通道维度 -> (60000, 1, 28, 28)

label = idx2numpy.convert_from_file(labelpath)
one_hot_labels = one_hot(label, 10)

In [6]:
epochs = 1
pbar =tqdm(range(epochs))

for i in pbar:
    data, one_hot_labels = shuffle(data, one_hot_labels)
    net.train(data, one_hot_labels, batch_size=64, lr=10, lossfunc="cross_entropy")

    y_hat = net.predict(data)
    loss = cross_entropy_loss(one_hot_labels, y_hat)
    y_hat = np.argmax(y_hat, axis=1)    # (60000, 10) -> (60000,)
    accuracy = np.mean(y_hat == one_hot_labels.argmax(axis=1))
    pbar.set_postfix({"loss": loss, "accuracy": f"{accuracy*100:.2f}%"})

  0%|          | 0/1 [00:00<?, ?it/s]

Batch 1/937, Loss: 2.3033
Batch 2/937, Loss: 2.5388
Batch 3/937, Loss: 4.3180
Batch 4/937, Loss: 4.1744
Batch 5/937, Loss: 4.1744
Batch 6/937, Loss: 4.3180
Batch 7/937, Loss: 4.2462
Batch 8/937, Loss: 3.9590
Batch 9/937, Loss: 4.2462
Batch 10/937, Loss: 3.9590
Batch 11/937, Loss: 4.3180
Batch 12/937, Loss: 4.3180
Batch 13/937, Loss: 4.1744
Batch 14/937, Loss: 4.2462
Batch 15/937, Loss: 4.1744
Batch 16/937, Loss: 4.0308
Batch 17/937, Loss: 4.1744
Batch 18/937, Loss: 4.2462
Batch 19/937, Loss: 3.9590
Batch 20/937, Loss: 4.2462
Batch 21/937, Loss: 3.7436
Batch 22/937, Loss: 4.3180
Batch 23/937, Loss: 4.2462
Batch 24/937, Loss: 4.1744
Batch 25/937, Loss: 4.1744
Batch 26/937, Loss: 3.8872
Batch 27/937, Loss: 4.1026
Batch 28/937, Loss: 4.2462
Batch 29/937, Loss: 3.8872
Batch 30/937, Loss: 4.1744
Batch 31/937, Loss: 4.3180
Batch 32/937, Loss: 4.2462
Batch 33/937, Loss: 4.0308
Batch 34/937, Loss: 4.3180
Batch 35/937, Loss: 4.3180
Batch 36/937, Loss: 4.1744
Batch 37/937, Loss: 4.1744
Batch 38/9

  0%|          | 0/1 [00:21<?, ?it/s]

Batch 60/937, Loss: 4.3898


KeyboardInterrupt: 

In [ ]:
#net.save_params(modelpath_2)